In [19]:
!pip install qiskit
!pip install qiskit[nature]

In [1]:
from qiskit_nature.drivers.second_quantization import PySCFDriver
from qiskit_nature.drivers import UnitsType
from qiskit_nature.second_q.transformers import FreezeCoreTransformer
from qiskit_nature.problems.second_quantization.electronic import ElectronicStructureProblem

driver = PySCFDriver(atom='Be 0 0 0; H 0 0 0.668; H 0 0 -0.668', unit=UnitsType.ANGSTROM, basis='sto3g')
problem = ElectronicStructureProblem(driver,[FreezeCoreTransformer(freeze_core=True,
                               )])
second_q_ops = problem.second_q_ops()
main_op = second_q_ops['ElectronicEnergy']
particle_number = problem.grouped_property_transformed.get_property("ParticleNumber")
num_particles = (particle_number.num_alpha, particle_number.num_beta)
num_spin_orbitals = particle_number.num_spin_orbitals

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [3]:
num_spin_orbitals

14

In [2]:
num_particles

(3, 3)

In [ ]:
from qiskit_nature.mappers.second_quantization import JordanWignerMapper
from qiskit_nature.converters.second_quantization import QubitConverter

mapper = JordanWignerMapper()
converter = QubitConverter(mapper=mapper, two_qubit_reduction=True)
qubit_op = converter.convert(main_op, num_particles=num_particles)

In [ ]:
from qiskit import BasicAer,Aer
from qiskit_nature.circuit.library import UCCSD
from qiskit.algorithms import VQE
from qiskit.algorithms.optimizers import GradientDescent
from qiskit_nature.circuit.library import HartreeFock

backend = BasicAer.get_backend("statevector_simulator")
optimizer = GradientDescent(maxiter=100)
init_state = HartreeFock(num_spin_orbitals, num_particles, converter)
var_form = UCCSD(converter,
                     num_particles,
                     num_spin_orbitals,
                     initial_state=init_state)
vqe = VQE(var_form, optimizer, quantum_instance=backend)
vqe_calc = vqe.compute_minimum_eigenvalue(qubit_op)
vqe_result = problem.interpret(vqe_calc).total_energies[0].real

In [ ]:
print(f"VQE Result: {vqe_result:.5f}")

In [ ]:
from qiskit_nature.second_q.algorithms import NumPyMinimumEigensolverFactory, GroundStateEigensolver
solver = NumPyMinimumEigensolverFactory()
calc = GroundStateEigensolver(converter, solver)
exact_result = calc.solve(problem).total_energies[0].real


In [ ]:
print(f"Exact result: , {exact_result: .5f}")